In [ ]:
import requests
import json

from flask import Flask, render_template
from flask import request
from wtforms import Form, TextAreaField, validators

'''
Client with web UI
'''
app = Flask(__name__)

class SearchForm(Form):
    query = TextAreaField('',[validators.DataRequired()])

@app.route('/')
def index():
    form = SearchForm(request.form)
    return render_template('search.html', form=form)

@app.route('/search', methods=['POST'])
def search():
    
    form = SearchForm(request.form)
    if request.method == 'POST' and form.validate():
        query = request.form['query']
        
        response_checker = requests.post('http://127.0.0.1:13539/spellchecker',
                                         json={'query': query})
        parsed_checker = json.loads(response_checker.text)
        
        if parsed_checker['state']:
            # if checker found mistakes in the query
            r = requests.post('http://127.0.0.1:13565/search', 
                              json={'query': parsed_checker["processed_data"], 
                                    'max_docs': 10}).json()["documents"]
            if len(r) == 0:
                return render_template('blank.html', old_query=query, form=form)
            title = [doc['title'] for doc in r]
            url = [doc['url'] for doc in r]
            snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]
    
            return render_template('checked.html', old_query=query,
                                   checked_query=parsed_checker["processed_data"],
                                   title=title, url=url, snippet=snippet, form=form)
        else:
            r = requests.post('http://127.0.0.1:13565/search', 
                              json={'query': query, 'max_docs': 10}).json()["documents"]
            if len(r) == 0:
                return render_template('blank.html', old_query=query, form=form)
        
            title = [doc['title'] for doc in r]
            url = [doc['url'] for doc in r]
            snippet = [doc['snippet'] +' (' + str(doc['id']) +')' for doc in r]

            return render_template('result.html', old_query=query, 
                                   query=query,title=title, url=url, 
                                   snippet=snippet, form=form)
    
    return render_template('search.html', form=form)

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=13560)